In [ ]:
from simtk.openmm.app import *
from simtk.openmm import *
from simtk.unit import *
from sys import stdout
import time
import mdtraj as md
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
prot = md.load_pdb('/home/eva/Documents/designs/p-IL-dimer/simulation-input/p-IL_fromtleap.pdb')
topology = prot.topology

In [ ]:
g1 = [atom.index for atom in topology.chain(0).atoms]
g2 = [atom.index for atom in topology.chain(1).atoms]

In [ ]:
coord = md.load('/home/eva/Documents/designs/p-IL-dimer/simulation-input/p-IL.inpcrd', top = '/home/eva/Documents/designs/p-IL-dimer/simulation-input/p-IL.prmtop')

In [ ]:
phi_array = md.compute_phi(coord)
psi_array = md.compute_psi(coord)

phi_atoms = phi_array[0].tolist()
psi_atoms = psi_array[0].tolist()

phi_atoms_res_5_25 = phi_atoms[4:25]+phi_atoms[34:55]
psi_atoms_res_5_25 = psi_atoms[4:25]+psi_atoms[34:55]

In [ ]:
start_time = time.time()

prmtop = AmberPrmtopFile('/home/eva/Documents/designs/p-IL-dimer/simulation-input/p-IL.prmtop')
inpcrd = AmberInpcrdFile('/home/eva/Documents/designs/p-IL-dimer/simulation-input/p-IL.inpcrd')
pdb = PDBFile('/home/eva/Documents/designs/p-IL-dimer/simulation-input/p-IL_fromtleap.pdb')

system = prmtop.createSystem(nonbondedMethod=NoCutoff, constraints=HBonds, hydrogenMass=1.5*amu, 
                             implicitSolvent=OBC2)
cvmin = 0.1
cvmax = 10.0

expr = 'distance(g1,g2)'
#in metadynamics, the expression inside the Force object needs to be the CV, not an energy expression

dist_force_1 = openmm.CustomCentroidBondForce(2, expr)
dist_force_1.addGroup(g1)
dist_force_1.addGroup(g2)
dist_force_1.addBond([0,1])
#dist_force.setForceGroup(1)
#do not add force to the system manually, it will get added in the metadynamics step as a CV force
dist_bias_1 = BiasVariable(dist_force_1, cvmin, cvmax, 0.05, False)

#average alpha helix phi and psi dihedrals
phi_avg = (-57.0*np.pi)/180.0
psi_avg = (-47.0*np.pi)/180.0

#Ramachandran plot region for right-handed alpha-helix (in deg)
phi_min = -130.0
phi_max = -30.0
psi_min = -68.0
psi_max = 30.0

phi0 = (phi_min +phi_max)/2.0
psi0 = (psi_min+psi_max)/2.0
phicutoff = abs(phi_max-phi_min)/2.0
psicutoff = abs(psi_max-psi_min)/2.0
        
phiforce = CustomTorsionForce('select({},{},{})'.format('step(-k_dihed*cos(theta-phi0)-(-k_dihed*cos_phicutoff))', '-k_dihed*cos(theta-phi0)', '-k_dihed*cos_phicutoff'))
phiforce.addGlobalParameter('k_dihed', 100.0*kilojoules_per_mole)
phiforce.addGlobalParameter('phi0', (phi0*np.pi/180.0)*radians)
phiforce.addGlobalParameter('cos_phicutoff', cos(phicutoff*np.pi/180.0))

psiforce = CustomTorsionForce('select({},{},{})'.format('step(-k_dihed*cos(theta-psi0)-(-k_dihed*cos_psicutoff))', '-k_dihed*cos(theta-psi0)', '-k_dihed*cos_psicutoff'))
psiforce.addGlobalParameter('k_dihed', 100.0*kilojoules_per_mole)
psiforce.addGlobalParameter('psi0', (psi0*np.pi/180.0)*radians)
psiforce.addGlobalParameter('cos_psicutoff', cos(psicutoff*np.pi/180.0))


for i in range(len(phi_atoms_res_5_25)):
    phiforce.addTorsion(phi_atoms_res_5_25[i][0], phi_atoms_res_5_25[i][1], phi_atoms_res_5_25[i][2], phi_atoms_res_5_25[i][3])
    psiforce.addTorsion(psi_atoms_res_5_25[i][0], psi_atoms_res_5_25[i][1], psi_atoms_res_5_25[i][2], psi_atoms_res_5_25[i][3])
    
phiforce.setForceGroup(2)
psiforce.setForceGroup(3)
system.addForce(phiforce)
system.addForce(psiforce)

upper_wall = CustomCentroidBondForce(2, 'k_upper*max(distance(g1,g2)-cvmax, 0)^2')
upper_wall.addGroup(g1)
upper_wall.addGroup(g2)
upper_wall.addBond([0,1])
upper_wall.addGlobalParameter('k_upper', 10000.0*kilojoules_per_mole/nanometers**2)
upper_wall.addGlobalParameter('cvmax', cvmax)
upper_wall.setForceGroup(4)
system.addForce(upper_wall)


meta = Metadynamics(system, [dist_bias_1], 298.15*kelvin, 15.0, 0.8*kilojoule_per_mole, 500)

integrator = LangevinMiddleIntegrator(298.15*kelvin, 1/picosecond, 0.004*picoseconds)

platform = Platform.getPlatformByName('CUDA')
properties = {'Precision': 'mixed'}

simulation = Simulation(prmtop.topology, system, integrator, platform, properties)

simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(DCDReporter('output.dcd', 1000))
simulation.reporters.append(StateDataReporter(stdout, 1000, step=True,
        potentialEnergy=True, temperature=True))

meta.step(simulation, 250000000)

print ("simulation time:", time.time() - start_time, "s")

In [ ]:
traj = md.load('output.dcd', top = '/home/eva/Documents/designs/p-IL-dimer/simulation-input/p-IL.prmtop')

cv_tuple=[]

for crd in traj.xyz: #getting atom coordinates in each frame to make the simulation context
    simulation.context.setPositions(crd)
    cv_tuple.append(meta.getCollectiveVariables(simulation)) #get CV per frame


In [ ]:
fe = []
for i in meta.getFreeEnergy():
    fe.append(i._value)

In [ ]:
f = open('cv.txt', 'w')
for i in cv_tuple:
    f.write(str(i[0])+'\n') 
f.close()

f2 = open('fe.txt', 'w')
for i in fe:
    f2.write(str(i)+'\n')
f2.close()

f3 = open('total-bias.txt', 'w')
for i in meta._totalBias:
    f3.write(str(i)+'\n')  
f3.close()